In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine,text 
import leafmap

In [3]:
from fgdbDL import download_and_extract_gdb 

url = "https://open.hkmapservice.gov.hk/OpenData/directDownload?productName=iB1000&sheetName=Fullset_Seamless&productFormat=FGDB"

storage_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1"
download_and_extract_gdb(url, storage_path)

Downloaded file to C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\downloaded_file.zip
Extracted 20240527 to C:\Users\Steve_Lau\Desktop\LS Training\ls_project1
Removed the original downloaded file C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\downloaded_file.zip


In [13]:
# Define the paths and layer name (comment out either one gdb_path when not in use)
#gdb_path = "/home/steeb/Documents/GIS/20240509/iB1000.gdb"
gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\20240527\iB1000.gdb"

# Define the db connection parameters
username = "postgres"
password = "12345"
host = "localhost"
dbname = "iB1000"
port = "5432"

pg_connection = f"PG:host={host} port={port} dbname={dbname} user={username} password={password}"
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

In [ ]:
from gdb2pgsql import transfer_gdb_to_postgis

transfer_gdb_to_postgis(gdb_path, pg_connection)

In [14]:
from gdbList import list_layers_with_types

list_layers_with_types(gdb_path)

Layers in the GeoDatabase:
Layer: UtilityNumber, Type: None
Layer: BUILDINGRELATEBUILDINGNAME, Type: None
Layer: SITENAME, Type: None
Layer: SITERELATESITENAME, Type: None
Layer: Street_Code, Type: None
Layer: BUILDINGNAME, Type: None
Layer: ADDRESSRELATEBUILDINGSITE, Type: None
Layer: ADDRESS, Type: None
Layer: BSLineAnno, Type: Multi Polygon
Layer: BSPointAnno, Type: Multi Polygon
Layer: BSPolygonAnno, Type: Multi Polygon
Layer: Building, Type: 3D Multi Polygon
Layer: BuildingAccessoryLine, Type: 3D Multi Line String
Layer: BuildingAnno, Type: Multi Polygon
Layer: BuildingAnnoHouseNo, Type: Multi Polygon
Layer: BuiltStructureLine, Type: 3D Multi Line String
Layer: BuiltStructurePoint, Type: 3D Point
Layer: BuiltStructurePolygon, Type: 3D Multi Polygon
Layer: CartoBuildingLine, Type: 3D Multi Line String
Layer: ProposedInfraAnno, Type: Multi Polygon
Layer: ProposedInfraStructureLine, Type: 3D Multi Line String
Layer: ProposedInfraStructurePoint, Type: 3D Point
Layer: Site, Type: 3D Mu

In [3]:
table_blgname = "BUILDINGNAME"
table_blg_blgname = "BUILDINGRELATEBUILDINGNAME"
table_blg = "building"
table_site = "site"

sql_blgname = text(f"SELECT * FROM {table_blgname}")
sql_blg_blgname = text(f"SELECT * FROM {table_blg_blgname}")
sql_blg = text(f"SELECT * FROM {table_blg}")
sql_site = text(f"SELECT * FROM {table_site}")

In [4]:
# Read the tables into DataFrames
df_blgname = pd.read_sql(sql_blgname, con=engine.connect())
df_blg_blgname = pd.read_sql(sql_blg_blgname, con=engine.connect())

# Read the tables with geometry into DataFrames
gdf_blg = gpd.read_postgis(sql_blg, con=engine.connect(), geom_col="shape") 
gdf_site = gpd.read_postgis(sql_site, con=engine.connect(), geom_col="shape")

In [6]:
# Select only the "BUILDINGNAMEID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df_blgname
df_blgname_subset = df_blgname[["buildingnameid",
                  "englishbuildingname",
                  "chinesebuildingname"]]

# Merge df_blg_blgname with the subset of df_blgname on the "BUILDINGNAMEID" column
df_merge_blgname = pd.merge(df_blg_blgname,
                            df_blgname_subset,
                            on="buildingnameid",
                            how="right")

In [7]:
# Select only the "BUILDINGID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df_merge_blgname
df_merge_blgname_subset = df_merge_blgname[["buildingid",
                  "englishbuildingname",
                  "chinesebuildingname"]]

# Merge gdf_blg with the subset of df_merge_blgname on the "BUILDINGID" column
gdf_merge_blgname = pd.merge(gdf_blg,
                df_merge_blgname_subset,
                on="buildingid", how="left")

In [8]:
# Drop duplicated rows based on "buildingid"
gdf_merge_blgname = gdf_merge_blgname.drop_duplicates(subset="buildingid")

# Keep only relevant columns
gdf_merge_blgname = gdf_merge_blgname.loc[:, ("buildingid",
                    "typeofbuildingblock",
                    "baselevel",
                    "rooflevel",
                    "buildingstatus",
                    "englishbuildingname",
                    "chinesebuildingname",
                    "shape")]

In [9]:
dict_col = {1:'Farm',
            2:'Accommodation',
            3:'Commerce',
            4:'Education And Training',
            5:'Health And Medical Service',
            6:'Leisure Culture And Sports',
            7:'Organization',
            8:'Public And Social Service',
            9:'Religion',
            10:'Tourist Attraction',
            11:'Military',
            12:'Industry',
            13:'Works Area',
            14:'Transportation',
            15:'Utilities'
            }

gdf_site['sitestype'] = gdf_site['sitestype'].map(dict_col)

In [10]:
gdf_overlay_blg_site = gpd.overlay(gdf_merge_blgname,
                                   gdf_site,
                                   how='intersection')
gdf_overlay_blg_site['area'] = gdf_overlay_blg_site.geometry.area

In [ ]:
gdf_overlay_blg_site.sort_values(by='area', inplace=True)
gdf_overlay_blg_site.drop_duplicates(subset='buildingid',
                                     keep='last',
                                     inplace=True)
gdf_overlay_blg_site.drop(columns=['area'], inplace=True)

In [ ]:
gdf_overlay_blg_site.head()

In [ ]:
gdf_overlay_sitetype = gdf_overlay_blg_site[["buildingid",
                                             "sitestype",
                                             "sitecode"]]

gdf_sjoin_blg_site = gdf_merge_blgname.merge(gdf_overlay_sitetype,
                                             on="buildingid",
                                             how="left")

gdf_sjoin_blg_site.head()

In [ ]:
# Keep only relevant columns
gdf_sjoin_blg_site = gdf_sjoin_blg_site.loc[:, ("buildingid",
                    "typeofbuildingblock",
                    "baselevel",
                    "rooflevel",
                    "buildingstatus",
                    "englishbuildingname",
                    "chinesebuildingname",
                    "sitestype",
                    "sitecode",
                    "shape")]

In [ ]:
shp_path_1 = "/home/steeb/Documents/GIS/buildings.shp"
gdf_merge_blgname.to_file(shp_path_1, encoding='utf-8')

In [ ]:
shp_path_2 = "/home/steeb/Documents/GIS/buildingsites_sjoin.shp"
gdf_sjoin_blg_site.to_file(shp_path_2, encoding='utf-8')

In [ ]:
m = leafmap.Map()

m.add_gdf(gdf_sjoin_blg_site, layer_name="buildings")
m